# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes
https://www.ctcrash.uconn.edu/QueryTool2.action

* Since the tool does not let you export large datasets (over 90,000 persons), export multiple times for different time periods and specify all export IDs in the `exports` list below.

This notebook combines data from two tables (0 and 2) into a small JSON file and a CSV file (same content, different formats), which is an array of objects that look like this:
```js
{
    "id": 770826,    // crash id (used to pull the diagram)
    "x": 41.73936,   // latitude
    "y": -72.66404,  // longitude
    "k": 1/1/2020    ///date for crash 
    "d": 1593820800, // timestamp
    "t": 09:41:00    // time of crash
    "s": 0,          //// Most Sever Injury
    "r": 1,          // route class (1=Interstate, 2=USRoute, 3=State, 4=Local)
    "p": 0,          // involves a pedestrian or other pedestrian? (1=true, 0=false)
    "c": 0,          // involves a cyclist or other cyclist? (1=true, 0=false)
    "to": 0          /// total other injuries (minor) 
    "ts": 0          /// total serious injuries
    "tf": 0          /// total fatal injuries
```

Note: the timestamp `d` needs to be multiplied by `100,000` for a proper JS timestamp.

In [21]:
# Importing in Panda Library used for data manipulation and analysis
import pandas as pd

In [22]:
# Setting name of folder that raw data is in
exports = ['40996']

### Import roles from table 2

In [23]:
total_dfs = []

# Iterating through export_xxxxx_2.csv (roles) file and creates new array of data only from CrashID and Person Type
for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_2.csv',
        skiprows=1,
        usecols=['CrashId', 'Injury Status'],
        encoding='ISO-8859-1'
    )    
    
    
    total_dfs.append( df ) # Adding two columns of data to new array 

totals = pd.concat(total_dfs).groupby(['CrashId'])['Injury Status'].unique() # Grouping data using two columns specified

In [24]:
totals.head(10)

CrashId
2000          [O]
2001          [O]
2002          [O]
2003          [O]
2004          [O]
2005     [nan, B]
2006          [O]
2007          [O]
2008    [O, C, B]
2009          [O]
Name: Injury Status, dtype: object

In [25]:
currentId = df['CrashId'][0]
col1 = []
col2 = []
countO = 0
for ind in df.index:
    if df['CrashId'][ind] == currentId:
        if df['Injury Status'][ind] == "O":
            countO = countO +  1
    else:
        col1.append(df['CrashId'][ind - 1])
        col2.append(countO)
        currentId = df['CrashId'][ind]
        if df['Injury Status'][ind] == "O":
            countO = 0
            countO = countO +  1
        else:
            countO = 0
#print(col1)
#print(col2)
d = {'CrashId': col1, 'Injury Status': col2 }
totalsO = pd.DataFrame(data=d)
print(totalsO)

       CrashId  Injury Status
0         2000              2
1         2001              1
2         2002              1
3         2003              2
4         2004              3
...        ...            ...
13923  1014560              2
13924  1014704              1
13925  1014706              3
13926  1015010              0
13927  1015532              0

[13928 rows x 2 columns]


In [26]:
currentId = df['CrashId'][0]
col1 = []
col2 = []
countK = 0
for ind in df.index:
    if df['CrashId'][ind] == currentId:
        if df['Injury Status'][ind] == "K":
            countK = countK +  1
    else:
        col1.append(df['CrashId'][ind - 1])
        col2.append(countK)
        currentId = df['CrashId'][ind]
        if df['Injury Status'][ind] == "K":
            countK = 0
            countK = countK +  1
        else:
            countK = 0
#print(col1)
#print(col2)
d = {'CrashId': col1, 'Injury Status': col2 }
totalsK = pd.DataFrame(data=d)
print(totalsK)

       CrashId  Injury Status
0         2000              0
1         2001              1
2         2002              1
3         2003              1
4         2004              1
...        ...            ...
13923  1014560              1
13924  1014704              1
13925  1014706              1
13926  1015010              0
13927  1015532              0

[13928 rows x 2 columns]


In [380]:
roles_dfs = []

# Iterating through export_xxxxx_2.csv (roles) file and creates new array of data only from CrashID and Person Type
for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_2.csv',
        skiprows=1,
        usecols=['CrashId', 'Person Type'],
        encoding='ISO-8859-1'
    )    
    roles_dfs.append( df ) # Adding two columns of data to new array 

roles = pd.concat(roles_dfs).groupby(['CrashId'])['Person Type'].unique()# Grouping data using two columns specified

In [358]:
roles.head(10)

CrashId
2000       [1]
2001       [1]
2002       [1]
2003       [1]
2004    [1, 2]
2005    [9, 1]
2006       [1]
2007       [1]
2008    [2, 1]
2009    [2, 1]
Name: Person Type, dtype: object

### Import crashes from table 0

In [359]:
crashes_dfs = []

# Iterating through export_xxxxx_0.csv (crashes) file and creates new array of data using following categories:
#'CrashId', 'Latitude', 'Longitude', 'Date Of Crash', 'Time of Crash', 'Most Severity', 'Route Class'
for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_0.csv',
        skiprows=1,
        usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash',
                 'Time of Crash', 'Most Severe Injury', 'Route Class']
    )
    df['Date Place Holder'] = df.loc[:, 'Date Of Crash'] # Adding duplicate column of date as last column to be used for new CSV
    df = df[['CrashId', 'Latitude', 'Longitude', 'Date Of Crash', 'Date Place Holder',
                 'Time of Crash', 'Most Severe Injury', 'Route Class']] # Rearranging Columns so that dates are next to eachother
    crashes_dfs.append( df ) 
    
   
    
crashes = pd.concat(crashes_dfs) # Grouping data using 8 columns specified 

### Merge data and save to .json and .csv

In [360]:
crashes.head() 

CrashId   Latitude  Longitude Date Of Crash Date Place Holder  \
0     2000  41.754402 -72.736591    2015-01-02        2015-01-02   
1     2001  41.724997 -72.759667    2015-01-01        2015-01-01   
2     2002  41.724880 -72.740740    2015-01-03        2015-01-03   
3     2003  41.762320 -72.740570    2015-01-03        2015-01-03   
4     2004  41.752930 -72.743672    2015-01-03        2015-01-03   

  Time of Crash Most Severe Injury  Route Class  
0      09:41:00                  O          4.0  
1      18:00:00                  O          NaN  
2      09:42:00                  O          4.0  
3      12:05:00                  O          4.0  
4      14:49:00                  O          4.0

### Filter Data From Exports 0 and 2 to create Smaller CSV and JSON File

In [361]:
# Rename columns to make JSON output smaller
crashes.columns = ['id', 'x', 'y', 'k', 'd', 't', 's', 'r']

# Transforming datestamp into timestap 
crashes.d = crashes.d.apply(lambda x: int(pd.to_datetime(x).timestamp() / 100))
crashes.t = crashes.t.apply(lambda x: x[:-3])

# Setting up pedestrian and cyclist flags to CSV, adds a 1(true) or 0(false) for pedestrain or cyclist column 
# 3 is person type code for Pedestrain, 4 is Other Pedestrain 
# 5 is person type code for Cyclist, 100 is placeholder for when we find Other Cyclist Code 
crashes['p'] = crashes.id.apply(lambda x: 1 if x in roles and ( 3 in roles[x] or 4 in roles[x]) else 0) 
crashes['c'] = crashes.id.apply(lambda x: 1 if x in roles and ( 5 in roles[x] or 100 in roles[x]) else 0)


#crashes['ts'] = crashes.id.apply(lambda x: + 1 if x in totals and ('A' in totals[x]) else 0)
#crashes['to'] = crashes.id.apply(lambda x: + 1 if x in totals and ('O' in totals[x]) else 0)
#crashes['to'] = crashes.id.apply(lambda x: + 1 if x in totals and ('B' in totals[x]) else 0)
#crashes['to'] = crashes.id.apply(lambda x: + 1 if x in totals and ('C' in totals[x]) else 0)



In [362]:
# Save as JSON
crashes.to_json('./crashes.json', orient='records', double_precision=5)

# Save as CSV
crashes.to_csv('crashes.csv', index=False)

In [363]:
crashes.head() 

id          x          y           k         d      t  s    r  p  c
0  2000  41.754402 -72.736591  2015-01-02  14201568  09:41  O  4.0  0  0
1  2001  41.724997 -72.759667  2015-01-01  14200704  18:00  O  NaN  0  0
2  2002  41.724880 -72.740740  2015-01-03  14202432  09:42  O  4.0  0  0
3  2003  41.762320 -72.740570  2015-01-03  14202432  12:05  O  4.0  0  0
4  2004  41.752930 -72.743672  2015-01-03  14202432  14:49  O  4.0  0  0

### Just custom calculations – not needed for processing